<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Cisternas
- Nombre de alumno 2: Diego González


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/jcist/MDS7202`

# Importamos librerias útiles

In [18]:
#!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\joaci\AppData\Local\Temp\ipykernel_22044\3184305967.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [20]:
# Inserte su código acá

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import pickle
import joblib

# fijar semilla 

np.random.seed(123)

#seed de sklearn

random_state = 123

# 1. Separa los datos
# Supongamos que 'df' es tu DataFrame y 'target' es tu variable objetivo
X = df.drop(['id', 'quantity'], axis=1)
y = df['quantity']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

# 2. Implementa un FunctionTransformer
def extract_date_info(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

date_transformer = FunctionTransformer(extract_date_info, validate=False)

# 3. Implementa un ColumnTransformer
numeric_features = X.select_dtypes(include=['float64']).columns
categorical_features = ['day', 'month', 'year', 'city', 'shop', 'brand', 'container', 'capacity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 4. Guarda los pasos en un Pipeline
pipeline_dummy = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy = 'mean'))
])

# 5. Entrenar el pipeline y reportar MAE en datos de validación
pipeline_dummy.fit(X_train, y_train)
y_val_pred_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_val_pred_dummy)
print(f'MAE con Dummy Regressor: {mae_dummy}')
print('-----------------------------------------------------------------------------------------------------------------')
print('Esta metrica para el contexto del negocio se interpreta como la distancia entre los valores predichos y')
print('los reales respecto a la cantidad de ventas. Específicamente en este caso nos indica que hay una diferencia')
print('promedio de 13308 ventas aproximadamente.')
print('-----------------------------------------------------------------------------------------------------------------')

# 6. Entrenar el Pipeline con XGBRegressor
pipeline_xgb = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor())
])

pipeline_xgb.fit(X_train, y_train)
y_val_pred_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f'MAE con XGBRegressor: {mae_xgb}')
print('-----------------------------------------------------------------------------------------------------------------')
print('Al implementar este algoritmo el MAE disminuye considerablemente, específicamente es mejor ya que el MAE es menor.')

# 7. Guardar ambos modelos en archivos .pkl
pipeline_dummy.fit(X_train, y_train)
pipeline_xgb.fit(X_train, y_train)

joblib.dump(pipeline_dummy, 'pipeline_dummy.pkl')
joblib.dump(pipeline_xgb, 'pipeline_xgb.pkl')


MAE con Dummy Regressor: 13308.134750658153
-----------------------------------------------------------------------------------------------------------------
Esta metrica para el contexto del negocio se interpreta como la distancia entre los valores predichos y
los reales respecto a la cantidad de ventas. Específicamente en este caso nos indica que hay una diferencia
promedio de 13308 ventas aproximadamente.
-----------------------------------------------------------------------------------------------------------------
MAE con XGBRegressor: 2370.393953168026
-----------------------------------------------------------------------------------------------------------------
Al implementar este algoritmo el MAE disminuye considerablemente, específicamente es mejor ya que el MAE es menor.


['pipeline_xgb.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [21]:
# Inserte su código acá

import xgboost as xgb

# Define el regresor XGB con restricción de monotonía
monotonic_constraints = '(0,0,0,0,0,0,0,0,0,0,0,-1)'
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', monotone_constraints=monotonic_constraints)

# Define el pipeline con el regresor XGB
pipeline_xgb_monotone = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_regressor)
])

# Entrena el modelo
pipeline_xgb_monotone.fit(X_train, y_train)

# Realiza predicciones en el conjunto de validación
y_val_pred_xgb_monotone = pipeline_xgb_monotone.predict(X_val)

# Calcula el MAE para evaluar el rendimiento
mae_xgb_monotone = mean_absolute_error(y_val, y_val_pred_xgb_monotone)
print(f'MAE con XGB Regressor (monotonicidad forzada): {mae_xgb_monotone}')

print('-----------------------------------------------------------------------------------------------------------------')
print('Al incluir esta relación el error disminuye pero no considerablemente, aún así podemos decir que el amigo tenía razón.')

# Guarda el modelo en un archivo .pkl
joblib.dump(pipeline_xgb_monotone, 'pipeline_xgb_monotone.pkl')

MAE con XGB Regressor (monotonicidad forzada): 2352.6104700473024
-----------------------------------------------------------------------------------------------------------------
Al incluir esta relación el error disminuye pero no considerablemente, aún así podemos decir que el amigo tenía razón.


['pipeline_xgb_monotone.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

**R:** 
- `learning_rate` controla la contribución de cada árbol al modelo y ajusta la importancia de cada árbol, un valor más bajo conduce generalmente a modelos más robustos. Hacen sentido los valores, ya que prueba con árboles con contribuciones practicamente nulas hasta contribuciones totales.
- `n_estimators` es el número de árboles a construir, un valor más alto generalmente mejora la capacidad del modelo para aprender patrones en los datos, pero aumenta el riesgo de sobreajuste. Hace sentido eñ rango, aunque talvez sea bueno considerar un valor máximo ligeramente mayor a 1000 ya que son 7000 datos, aunque debe ser un valor tal que se evite el sobreajuste.
- `max_depth` es la máxima profundidad de cada árbol, controla la complejidad de cada árbol individual. Un valor más alto permite al modelo aprender patrones más complejos, pero también puede conducir a sobreajuste. Hace sentido el rango, ya que los patrones aprendidos no serán tan complejos, evitando el sobreajuste.
-  `max_leaves` es el número máximo de hojas para cada árbol. Es útil para evitar que los árboles crezcan y se genere un sobreajuste. El rango hace sentido pues, dada la cantidad de datos, es un rango considerable de hojas que puede poseer cada árbol.
- `min_child_weight` controla la cantidad mínima de instancias necesarias en un nodo hijo, puede ser utilizado para controlar la sensibilidad del algoritmo a instancias individuales y reducir el sobreajuste. Los valores del rango hacen sentido.
- `reg_alpha` y `reg_lambda` son términos de regularización. `reg_alpha` controla la penalización L1 (lasso), mientras que `reg_lambda` controla la penalización L2 (ridge). Estos términos de regularización ayudan a prevenir el sobreajuste al agregar una penalización a los pesos de las características.

In [22]:
import optuna
from sklearn.pipeline import Pipeline
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import time
from functools import partial

In [23]:
X = df.drop(['id', 'quantity'], axis=1)
y = df['quantity']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)
seed = 123

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    # Definir el modelo con los hiperparámetros sugeridos por Optuna
    model = xgb.XGBRegressor(**params, random_state=seed)

    min_freq = {'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)}

    # Definir OneHotEncoder con el hiperparámetro a optimizar
    transformer = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(**min_freq), categorical_features),
            ('num', 'passthrough', numeric_features)
        ]
    )

    pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', transformer), 
        ('model', model)])
    
    start_time = time.time()
    
    # Entrenar el modelo con el pipeline
    pipeline.fit(X_train, y_train)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    
    end_time = time.time()

    # Fijar el tiempo de entrenamiento a 5 minutos
    if end_time - start_time > 300:
        raise optuna.exceptions.TrialPruned()

    return(mae)

# Configurar el sampler de Optuna
sampler = TPESampler(seed=seed)

# Crear función parcial con los datos de entrada
objective_partial = partial(objective)

# Configurar y ejecutar Optuna
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)

# Obtener los mejores hiperparámetros
best_params = study.best_params

# Imprimir resultados
print(f'Número de trials: {len(study.trials)}')
print(f'MAE: {study.best_value}')
print('Mejores hiperparámetros:')
print(best_params)
print('-----------------------------------------------------------------------------------------------------------------')
print('Respecto a la sección anterior el MAE disminuye considerablemente. Esto se puede deber al reajuste que hace el modelo de los hiperparámetros')
print('hasta encontrar la mejor combinación posible en los 5 minutos.')
print('-----------------------------------------------------------------------------------------------------------------')

Número de trials: 183
MAE: 1846.692301812573
Mejores hiperparámetros:
{'learning_rate': 0.08550747441225102, 'n_estimators': 907, 'max_depth': 9, 'max_leaves': 88, 'min_child_weight': 4, 'reg_alpha': 0.022765496663446995, 'reg_lambda': 0.11819470474682733, 'min_frequency': 0.06801574963443192}
-----------------------------------------------------------------------------------------------------------------
Respecto a la sección anterior el MAE disminuye considerablemente. Esto se puede deber al reajuste que hace el modelo de los hiperparámetros
hasta encontrar la mejor combinación posible en los 5 minutos.
-----------------------------------------------------------------------------------------------------------------


In [24]:
# Guardar el modelo en un archivo .pkl
best_param_onehot = best_params.pop('min_frequency')
best_model = xgb.XGBRegressor(**best_params, random_state=seed)
transformer = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(min_frequency=best_param_onehot), categorical_features),
            ('num', 'passthrough', numeric_features)
        ]
    )
best_pipeline = Pipeline(steps=[('preprocessor', transformer), ('model', best_model)])
best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, 'mejor_modelo.pkl')

['mejor_modelo.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

El prunning (poda) es una técnica de reducción de complejidad de los modelos de machine learning, eliminando nodos o ramas completas, esto se hace con la finalidad de hacer que los modelos puedan generalizar mejor, evitando el sobreajuste de los parámetros. Afecta al momento del entrenamiento en dos puntos: primero, permite una convergencia más rápida al óptimo debido a la reducción de parámetros y segundo, permite que el entrenamiento no sobreajuste los parámetros del modelo para que al testear la diferencia de rendimiento no sea tan alta

In [25]:
# Inserte su código acá

# hacer prunning con optuna.integrations.XGBoostPruningCallback

from optuna.integration import XGBoostPruningCallback

In [26]:
import time
import optuna
from optuna.integration import XGBoostPruningCallback
from functools import partial
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import OneHotEncoder

optuna.logging.set_verbosity(optuna.logging.WARNING)

# Definir la función objetivo con el uso de XGBoostPruningCallback
def objective(trial):
    seed = 123

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    min_freq = {'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)}

    model = xgb.XGBRegressor(**params, random_state=seed, eval_metric='mae', callbacks=[XGBoostPruningCallback(trial, 'validation_0-mae')])

    transformer = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(**min_freq), categorical_features),
            ('num', 'passthrough', numeric_features)
        ]
    )

    pipeline = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', transformer), 
        ('model', model)])
    
    # aplicar date_transformer y preprocessor a X_val

    X_val_transformed = date_transformer.transform(X_val)
    X_val_transformed = pipeline.named_steps['preprocessor'].fit(X_train).transform(X_val)

    start_time = time.time()
    
    # Entrenar el modelo con el pipeline y usar el callback de pruning

    evals = [(X_val_transformed, y_val)]

    pipeline.fit(
        X_train, y_train,
        model__eval_set=evals
    )
    
    end_time = time.time()

    if end_time - start_time > 300:
        raise optuna.exceptions.TrialPruned()

    # Evaluar el modelo en el conjunto de prueba
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    return mae

# Configurar el sampler de Optuna
sampler = optuna.samplers.TPESampler(seed=seed)

# Crear función parcial con los datos de entrada
objective_partial = partial(objective)

# Configurar y ejecutar Optuna con XGBoostPruningCallback
study_pruning = optuna.create_study(direction='minimize', sampler=sampler)
study_pruning.optimize(objective, timeout=300, show_progress_bar = True)

# Obtener los mejores hiperparámetros
best_params = study_pruning.best_params

# Imprimir resultados
print(f'Número de trials: {len(study_pruning.trials)}')
print(f'MAE: {study_pruning.best_value}')
print('Mejores hiperparámetros:')
print(best_params)


   0%|          | 00:00/05:00

[0]	validation_0-mae:12777.87481
[1]	validation_0-mae:12316.25753
[2]	validation_0-mae:11899.54654
[3]	validation_0-mae:11532.27933
[4]	validation_0-mae:11194.38262
[5]	validation_0-mae:10898.14819
[6]	validation_0-mae:10639.27990
[7]	validation_0-mae:10422.64188
[8]	validation_0-mae:10222.44362
[9]	validation_0-mae:10044.13502
[10]	validation_0-mae:9890.13372
[11]	validation_0-mae:9739.63940
[12]	validation_0-mae:9603.57609
[13]	validation_0-mae:9477.97155
[14]	validation_0-mae:9369.36260
[15]	validation_0-mae:9264.17885
[16]	validation_0-mae:9175.08670
[17]	validation_0-mae:9105.12674
[18]	validation_0-mae:9024.40006
[19]	validation_0-mae:8962.53267
[20]	validation_0-mae:8906.20710
[21]	validation_0-mae:8848.86916
[22]	validation_0-mae:8803.93625
[23]	validation_0-mae:8753.88388
[24]	validation_0-mae:8710.58429
[25]	validation_0-mae:8679.45971
[26]	validation_0-mae:8643.98937
[27]	validation_0-mae:8617.60594
[28]	validation_0-mae:8584.45578
[29]	validation_0-mae:8555.93241
[30]	valid

Los resultados cambian en relación al planteamiento anterior pues al podar se generaliza más el rendimiento del modelo, es decir, puede que no reduzca más el MAE, pero si tendrá un mejor desempeño en conjuntos de datos externos al entrenamiento y validación, reduciendo el overfitting. 

In [27]:
# Guardar el modelo en un archivo .pkl

best_param_onehot = best_params.pop('min_frequency')
best_model = xgb.XGBRegressor(**best_params, random_state=seed)
transformer = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(min_frequency=best_param_onehot), categorical_features),
            ('num', 'passthrough', numeric_features)
        ]
    )

best_pipeline = Pipeline(steps=[('preprocessor', transformer), ('model', best_model)])
best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, 'mejor_modelo_pruning.pkl')

['mejor_modelo_pruning.pkl']

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [28]:
# Inserte su código acá

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

SEED = 123

np.random.seed(SEED)

plot_optimization_history(study)

In [29]:
plot_parallel_coordinate(study)

In [30]:
plot_param_importances(study)

Se puede ver que para el modelo sin prunning, los resultados mejoran considerablemente a partir del trial 12, donde el MAE se reduce de 9000 a 2500, luego las siguientes iteraciones mejoran levemente los resultados, lo cual puede interpretarse como overfitting. Por otra parte, el hiperparámetro más importante es `min_frecuency`, explicando el 99% de la mejora del rendimiento 

In [31]:
plot_optimization_history(study_pruning)

In [32]:
plot_parallel_coordinate(study_pruning)

In [33]:
plot_param_importances(study_pruning)

Al hacer pruning, el ajuste de hiperparámetros es mucho más lento debido a la penalización al sobreajuste, es por esto que pasadas las 400 trial se ve un descenso importante. Por otra parte, ahora el parámetro `reg_lambda` tiene una importancia de 0.02, sin embargo `min_frequency` sigue siendo mucho más determinante con un 0.97

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [34]:
# generar tabla resumen de los resultados

import pandas as pd

summary = pd.DataFrame({'Modelo': ['Dummy Regressor', 'XGBRegressor', 'XGBRegressor (monotonicidad forzada)', 'XGBRegressor (Optuna)', 'XGBRegressor (Optuna + Pruning)'],
                        'MAE': [mae_dummy, mae_xgb, mae_xgb_monotone, study.best_value, study_pruning.best_value]})

summary

,Modelo,MAE
0,Dummy Regressor,13308.134751
1,XGBRegressor,2370.393953
2,XGBRegressor (monotonicidad forzada),2352.610470
3,XGBRegressor (Optuna),1846.692302
4,XGBRegressor (Optuna + Pruning),5590.272145


In [35]:
# cargar el mejor modelo y predecir sobre el conjunto de prueba

best_pipeline = joblib.load('mejor_modelo_pruning.pkl')

#transformar X_test

X_test_transformed = date_transformer.transform(X_test)
X_test_transformed = best_pipeline.named_steps['preprocessor'].fit(X_train).transform(X_test)

y_pred = best_pipeline.predict(X_test)

# calcular el MAE sobre el conjunto de prueba

mae = mean_absolute_error(y_test, y_pred)

print(f'MAE sobre el conjunto de prueba: {mae}')

MAE sobre el conjunto de prueba: 5734.39623461815


In [36]:
# cargar modelo sin prunning y predecir sobre el conjunto de prueba

best_pipeline = joblib.load('mejor_modelo.pkl')

#transformar X_test

X_test_transformed = date_transformer.transform(X_test)
X_test_transformed = best_pipeline.named_steps['preprocessor'].fit(X_train).transform(X_test)

y_pred = best_pipeline.predict(X_test)

# calcular el MAE sobre el conjunto de prueba

mae = mean_absolute_error(y_test, y_pred)

print(f'MAE sobre el conjunto de prueba: {mae}')

MAE sobre el conjunto de prueba: 1921.2288483037678


Ocurren diferencias en relación a los valores en el conjunto de validación, esto puede deberse a la naturaleza aleatoria de la división de datos, la cual provoca que el modelo no conozca los datos de prueba en ningún momento, esto ayuda a generalizar las predicciones, a cambio de reducir la presición exacta para una muestra. Otros factores que pueden ser relevantes es representatividad de las ventas, las cuales tienen distintas magnitudes dependiendo del producto, o también por problemas de sobreajuste.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>